In [1]:
import gc
import glob

import geopandas as gpd
import momepy as mm

import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [2]:
regions_buildings_dir = '/data/uscuni-ulce/regions/buildings/'
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
overture_streets_dir = '/data/uscuni-ulce/overture_streets/'
streets_dir = '/data/uscuni-ulce/processed_data/streets/'
enclosures_dir = '/data/uscuni-ulce/processed_data/enclosures/'
tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'
simplfied_buildings_dir = '/data/uscuni-ulce/processed_data/simplified_buildings/'


regions_datadir = "/data/uscuni-ulce/"
eubucco_files = glob.glob(regions_datadir + "eubucco_raw/*")

In [3]:
# df = pd.read_parquet('/data/uscuni-ulce/regions/cadastre_id_to_region.parquet')
# df.region.value_counts()
# '''
# region
# 4.0         11084489
# 107131.0     3078558
# 16242.0      1457482
# 4214.0       1389195
# 8707.0       1220411
#               ...   
# 7727.0         12682
# 6741.0         12235
# 10970.0        12170
# 10847.0        10871
# 15545.0        10409
# Name: count, Length: 828, dtype: int64



# df.region.value_counts().iloc[:20].index.values.astype(int)

# array([     4, 107131,  16242,   4214,   8707,  51289, 114311,  55763,
#         14679,  62954,  10579,  38679,  86873,  34266,     10,  80404,
#         18557,  69333,   1154, 130658])
# '''

##

In [4]:
# 107131 - krakow
# 86873 = vienna
# 69333 = prague
# munich
# 4 - rhineruhr
# 55763 - berlin
#bratislava - 99886
# freiburg - 5883
 # hamburg - 16242
# munich - 38679



# lithuania - 139196

In [5]:
region_name = 69333
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )
region_id, region_hull = region_hulls.loc[region_name].name, region_hulls.loc[region_name].convex_hull

In [6]:
# region_hulls.explore()

## Buildings


In [8]:
from core.generate_buildings import read_region_buildings, process_region_buildings

In [9]:
%%time

buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')

CPU times: user 2.02 s, sys: 1.25 s, total: 3.26 s
Wall time: 2.24 s


In [10]:
%%time

buildings = process_region_buildings(buildings, True, simplification_tolerance=.1, merge_limit=25)

Percent polygons:  1.0
Final polygons:  2052576 , dropped:  0.00026934498514219474
CPU times: user 2min 26s, sys: 1.73 s, total: 2min 28s
Wall time: 2min 28s


In [11]:

# orig_buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')
# dropped_buildings = orig_buildings[~orig_buildings['iid'].isin(buildings['iid'])]


# m = dropped_buildings.explore(color='red')
# m = buildings.explore(m=m)
# m

In [12]:
# buildings.to_parquet(simplfied_buildings_dir + f"buildings_{region_id}.parquet")

In [13]:
buildings.to_parquet(buildings_dir + f"buildings_{region_id}.parquet")

## Streets

In [14]:
from core.generate_streets import process_region_streets, process_single_region_streets

In [15]:
%%time
## processs streets
streets = process_region_streets(region_id, overture_streets_dir, buildings_dir)

/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:551: UserWarning: An error occured at location POINT (5275477.613340486 3781566.0414011283). The artifact has not been simplified. The original message:
can only convert an array of size 1 to a Python scalar
  roads = simplify_singletons(
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/coins.py:103: UserWarning: Lines are between points dict_keys([(5065036.230081792, 3785697.8965024077), (5065024.494818296, 3785711.2228520997)]) identical. Please revise input data to ensure no lines are identical or overlapping. You can check for duplicat

CPU times: user 43min 5s, sys: 7.39 s, total: 43min 12s
Wall time: 43min 5s


In [16]:
# save streets
streets.to_parquet(streets_dir + f'streets_{region_id}.parquet')

In [17]:
# streets.explore()

In [17]:
# import geopandas as gpd
# gpd.read_parquet(streets_dir + f'streets_{region_id}.parquet').explore()

In [18]:
# gpd.read_parquet(enclosures_dir + f'enclosure_{region_id}.parquet').explore()

## Enclosures & Tessellation

In [18]:
from core.generate_elements import process_region_elements, generate_enclosures_representative_points, generate_tess

In [19]:
# 21894
# 47090 - problems with topolgical exceptions

In [20]:
### copy the original buildings
# !cp /data/uscuni-ulce/processed_data/simplified_buildings/buildings_69333_merged.parquet /data/uscuni-ulce/processed_data/buildings/buildings_69333.parquet

In [21]:
%%time
enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)

---- Processing region:  139196 2024-11-29 13:22:16.418225
CPU times: user 9min 40s, sys: 13.4 s, total: 9min 53s
Wall time: 12min 41s


In [22]:
enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
print("Processed enclosures")

## save files
tesselations.to_parquet(
    tessellations_dir + f"tessellation_{region_id}.parquet"
)
print("processed tesselations")

Processed enclosures
processed tesselations


In [23]:
# import lonboard
# layeasr = lonboard.SolidPolygonLayer.from_geopandas(tesselations, opacity=0.15)
# m = lonboard.Map([layer])
# m

In [24]:
# layer = lonboard.SolidPolygonLayer.from_geopandas(enclosures, opacity=0.15)
# m = lonboard.Map([layer])
# m

## Graphs

In [25]:
from core.generate_ngraphs import process_region_graphs

In [26]:
%%time
process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
)

Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 415 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built nodes graph knn=1
CPU times: user 3min 57s, sys: 4.77 s, total: 4min 2s
Wall time: 3min 59s


In [27]:
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(graph_dir + '*')]

In [29]:
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(graph_dir + '*')]
region_hulls = region_hulls[~region_hulls.index.isin(processed_region_ids)]
region_hulls

,convex_hull
labels,


In [30]:
# from joblib import Parallel, delayed
# n_jobs = -1
# new = Parallel(n_jobs=n_jobs)(
#     delayed(process_region_graphs)(     region_id,
#                                         graph_dir,
#                                         buildings_dir,
#                                         streets_dir,
#                                         enclosures_dir,
#                                         tessellations_dir
#                                   ) 
#     for region_id, _ in region_hulls.iterrows())

In [31]:
1

1

## Characters

In [5]:
from core.generate_chars import process_single_region_chars

In [ ]:
%%time
process_single_region_chars(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    chars_dir
)

2024-11-29 13:47:42.509994 ----Processing ------ 139196
Processing streets


In [ ]:
from core.generate_chars import process_single_region_chars, process_street_chars, process_building_chars, process_tessellation_chars,process_enclosure_chars

In [31]:
# process_building_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [32]:
# %%time
# process_street_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [33]:
# process_tessellation_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [34]:
# process_enclosure_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

## Generate primary data

In [ ]:
from core.generate_merged_primary_chars import merge_into_primary

In [ ]:
%%time
merge_into_primary(region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    chars_dir)


## Generate morphotopes

In [7]:
from core.generate_clusters import process_single_region_morphotopes

In [8]:
 process_single_region_morphotopes(region_id,
            graph_dir,
            buildings_dir,
            streets_dir,
            enclosures_dir,
            tessellations_dir,
            chars_dir, morphotopes_dir)

2024-12-02 17:10:29.377606 ----Processing ------ 69333
--------Generating lag----------
--------Generating morphotopes---------- 75
--------Dropping columns---------- ['stcSAl', 'stbOri', 'stcOri', 'stbCeA', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB', 'likWBB', 'likWCe']
--------Generating morphotopes data----------


## Process multiple regions

In [5]:
from core.generate_buildings import read_region_buildings, process_region_buildings
from core.generate_streets import process_region_streets, process_single_region_streets
from core.generate_elements import process_region_elements, generate_enclosures_representative_points, generate_tess
from core.generate_ngraphs import process_region_graphs
from core.generate_chars import process_single_region_chars
from core.generate_merged_primary_chars import merge_into_primary
from core.generate_clusters import process_single_region_morphotopes

In [6]:
# # lithuania
# regions_to_process = [139621, 139674, 139764, 140420, 141272, 141366, 142100, 143701, 145906,
#        146285, 147112, 147634, 148018, 148085, 149997, 150044, 151676, 152081,
#        152511]


# ## czech socialist
# regions_to_process = [ 55835,  61183,  61187,  63756,  64463,  66593,  67156,  67803,
#         69333,  72507,  73735,  75472,  75642,  83538,  84297,  84962,
#         93167,  97978, 100210, 102127, 103882, 105090, 107131]

In [ ]:
for region_id in regions_to_process:

    print(region_id)
    # # buildings
    # buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')
    # buildings = process_region_buildings(buildings, True, simplification_tolerance=.1, merge_limit=25)
    # buildings.to_parquet(buildings_dir + f"buildings_{region_id}.parquet")

    ## processs streets
    streets = process_region_streets(region_id, overture_streets_dir, buildings_dir)
    streets.to_parquet(streets_dir + f'streets_{region_id}.parquet')

    # elements
    enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)
    enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
    tesselations.to_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
    )

    #graphs
    process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    )

    #chars
    process_single_region_chars(
        region_id,
        graph_dir,
        buildings_dir,
        streets_dir,
        enclosures_dir,
        tessellations_dir,
        chars_dir
    )

    # #primary merge
        merge_into_primary(region_id,
            graph_dir,
            buildings_dir,
            streets_dir,
            enclosures_dir,
            tessellations_dir,
            chars_dir)
    
        # # morphotope
        process_single_region_morphotopes(region_id,
            graph_dir,
            buildings_dir,
            streets_dir,
            enclosures_dir,
            tessellations_dir,
            chars_dir, morphotopes_dir)
        

55835


/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:551: UserWarning: An error occured at location POINT (4523880.794008707 3016718.189207473). The artifact has not been simplified. The original message:
can only convert an array of size 1 to a Python scalar
  roads = simplify_singletons(
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/coins.py:103: UserWarning: Lines are between points dict_keys([(4528156.877164154, 2995158.302805023), (4528157.597460956, 2995157.4556466816)]) identical. Please revise input data to ensure no lines are identical or overlapping. You can check for duplicates using `gdf.geometry.duplicated()`. Assumingan angle of 0 degrees.
  self._best_link()
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/coins.py:103: UserWarning: Lines are between points dict_keys([(4528158.317757758, 2995156.60848834), (4528157.597460956, 2995157.4556466816)]) identical. Please revise input data to ensure no lines are identical or overlapping. You can check fo

---- Processing region:  55835 2024-12-07 16:37:07.649703
Retrying tesselation with less buildings, potentially changing building data.
Dropping 2 buildings due to tesselation problems
Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 29 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built nodes graph knn=1
2024-12-07 16:38:03.995707 ----Processing ------ 55835
Processing streets
Processing enclosures
Processing buildings


...

---- Processing region:  67156 2024-12-07 16:58:19.501077
Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 32 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


Built nodes graph knn=1
2024-12-07 16:58:37.838023 ----Processing ------ 67156
Processing streets
Processing enclosures
Processing buildings


/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing tessellation
Processing region 67156
2024-12-07 16:59:47.987095 ----Processing ------ 67156
--------Generating lag----------
--------Generating morphotopes---------- 75
--------Dropping columns---------- ['stcSAl', 'stbOri', 'stcOri', 'stbCeA', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB', 'likWBB', 'likWCe']
--------Generating morphotopes data----------
67803
